## Set the enviroments.

#### Connect to slime.

In [1]:
(start-swank)

T

In [2]:
1

1

#### Add paths.

In [3]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber16/dat/leap/cmd/")
(add-path "~/Development/amber16/dat/leap/lib/")
(add-path "~/Development/amber16/dat/leap/parm/")
(add-path "~/Development/amber16/dat/leap/prep/")
(add-path "~/Development/amber16/dat/antechamber/")

(#P"/Users/tuj24515/Development/amber16/dat/antechamber/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/prep/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/parm/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/lib/"
 #P"/Users/tuj24515/Development/amber16/dat/leap/cmd/"
 #P"/Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [4]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [5]:
(source "leaprc.protein.ff14SB")

SIMPLE-WARNING: 
add-atom-types doesn't do anything
SIMPLE-WARNING: 
Warning!  Skipping 10-12 hbond
SIMPLE-WARNING: 
Warning!  Skipping force field atom equivalences


T

In [6]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [7]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [8]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [9]:
(build-hydrogens *chemagg*)

NIL

In [10]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

#### Gather atoms to the center.

In [11]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

C_0    S


NIL

#### Set the bounding box.

In [12]:
(chem:set-property *chemagg* :bounding-box '(50.0 50.0 50.0))

NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [13]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [14]:
(cando:jostle *chemagg*)

NIL

In [15]:
(defparameter *ee* (energy:minimize *chemagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minSDnP       0     1      -inf   0.0              0.000          17227.712
 minSDnP       0     2     -8.05   0.0        2729989.685           8620.324
 minSDnP       0     3     -8.05   0.0        2129818.209           5451.051
DONE absolute force test:
forceRmsMag(4559.632157).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
 minCGnP       0     4      -inf   0.0              0.000           4559.632
 minCGnP       0     5     -8.08  36.5        1546543.314           3372.217
 minCGnP       0     6     -6.78  46.1        1106628.072           3453.684
 minCGnP       0     7     -7.91  49.3         976908.373           2726.410
 minCGnP       0     8     -8.43  61.7         916655.576           3844.717
 minCGnP       0     9     -8.49  54.3         856640.244           2764.179
 minCGnP       0    10     -7.36  60.4   

*EE*

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [16]:
(nglv::show-aggregate *chemagg*)

A Jupyter Widget

In [17]:
(save-mol2 *chemagg* "alanine.mol2")

Saving matter to /Users/tuj24515/Development/clasp/extensions/cando/src/tests/alanine-demo/alanine.mol2


NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [18]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [19]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg.top" "chemagg.crd" :assign-types nil)

atom1: #<ATOM :C/:C> atom2: #<ATOM :C_0/:C> i1: 0 i2: 3
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 6 i2: 0
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 9 i2: 30
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C_0/:C> i1: 9 i2: 3
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :HH/:H> i1: 9 i2: 27
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C_0/:C> i1: 12 i2: 3
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C_0/:C> i1: 15 i2: 3
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 18 i2: 15
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 21 i2: 15
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 24 i2: 15
 atom1: #<ATOM :C/:C> atom2: #<ATOM :CH3/:C> i1: 30 i2: 33
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH31/:H> i1: 33 i2: 39
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH32/:H> i1: 33 i2: 42
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :HH33/:H> i1: 33 i2: 45
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 36 i2: 30
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 48 i2: 51
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C/:C> i1: 51

0

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [20]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [21]:
(print ~tip3pfbox)


#<AGGREGATE NIL> 

#<AGGREGATE NIL>

In [22]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [23]:
(nglv::show-aggregate *solvent*)

A Jupyter Widget

#### Assign the atom types of the force field to the water box.

In [24]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [25]:
(:= *newagg* (leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(5.0 5.0 5.0)))

Total bounding box for atom centers:  20.792837322611035d0 15.369380487777903d0 14.421193787284118d0


*NEWAGG*

In [26]:
(nglv::show-aggregate *newagg*)

A Jupyter Widget

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [27]:
(assign-atom-types *newagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *newagg*)

NIL

In [29]:
(defparameter *e* (energy:minimize *newagg*))

minimize-no-fail
======= Starting Steepest Descent Minimizer
DONE absolute force test:
forceRmsMag(59.709070).LT.forceTolerance(5000.000000)
======= Starting Conjugate Gradient Minimizer
---Stage-Seconds--Step-log(Alpha)--Dir-------------Energy-----------RMSforce
 minCGnP       0     1      -inf   0.0              0.000             59.709
 minCGnP       0     2     -7.38  36.4            796.232             44.059
 minCGnP       0     3     -7.21  27.2            200.626             18.111
 minCGnP       0     4     -7.34  29.6            110.944              8.890
 minCGnP       0     5     -6.89  41.1             77.287              6.657
 minCGnP       0     6     -7.09  45.9             61.754              4.959
 minCGnP       0     7     -6.53  55.9             46.673              5.078
 minCGnP       0     8     -6.67  50.7             32.989              3.371
 minCGnP       0     9     -6.97  56.1             28.516              3.173
 minCGnP       0    10     -6.98  55.9     

*E*

In [30]:
(leap.topology:save-amber-parm-format *newagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil)

atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 0 i2: 51
 atom1: #<ATOM :C_0/:C> atom2: #<ATOM :C/:C> i1: 3 i2: 0
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 6 i2: 0
 atom1: #<ATOM :NH/:N> atom2: #<ATOM :C_0/:C> i1: 9 i2: 3
 atom1: #<ATOM :H/:H> atom2: #<ATOM :C_0/:C> i1: 12 i2: 3
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C_0/:C> i1: 15 i2: 3
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 18 i2: 15
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 21 i2: 15
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 24 i2: 15
 atom1: #<ATOM :HH/:H> atom2: #<ATOM :NH/:N> i1: 27 i2: 9
 atom1: #<ATOM :C/:C> atom2: #<ATOM :NH/:N> i1: 30 i2: 9
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :C/:C> i1: 33 i2: 30
 atom1: #<ATOM :O/:O> atom2: #<ATOM :C/:C> i1: 36 i2: 30
 atom1: #<ATOM :HH31/:H> atom2: #<ATOM :CH3/:C> i1: 39 i2: 33
 atom1: #<ATOM :HH32/:H> atom2: #<ATOM :CH3/:C> i1: 42 i2: 33
 atom1: #<ATOM :HH33/:H> atom2: #<ATOM :CH3/:C> i1: 45 i2: 33
 atom1: #<ATOM :CH3/:C> atom2: #<ATOM :NH/:N> i1: 48 

0

#### Calculate each energy parameters.

In [31]:
(chem:get-energy (chem:get-stretch-component *e*))

1720.1189821006878d0

In [32]:
(chem:get-energy (chem:get-angle-component *e*))

0.30171081698364277d0

In [33]:
(chem:get-energy (chem:get-dihedral-component *e*))

5.456751705606037d0

In [34]:
(chem:get-energy (chem:get-nonbond-component *e*))

658.8024508583202d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL

In [88]:
(:= w (show *chemagg*)) w

A Jupyter Widget

In [93]:
(nglv::picked w)

(("atom" ("index" . 20) ("residueIndex" . 2) ("resname" . "NIL_3")
  ("x" . 216.329) ("y" . 322.549) ("z" . -0.62) ("element" . "H")
  ("chainname" . "A") ("resno" . 3) ("serial" . 21) ("vdw" . 1.1)
  ("covalent" . 0.31) ("hetero" . 1) ("bfactor" . 0) ("altloc" . "")
  ("atomname" . "HH33")))

In [94]:
(defun distance (p1 p2) (geom:vlength (geom:v- p1 p2)))

DISTANCE

In [95]:
(:= *a* (let* ((picked (nglv::picked w))
       (atom (cdr (assoc "atom" picked :test #'string=)))
       (x (cdr (assoc "x" atom :test #'string=)))
       (y (cdr (assoc "y" atom :test #'string=)))
       (z (cdr (assoc "z" atom :test #'string=)))
       (pos (geom:vec x y z))
        resulta)
     (chem:map-atoms nil (lambda (a) 
                                 (when (< (distance pos (chem:get-position a)) 0.01) 
                                     (setf resulta a))) *chemagg*)
              resulta))

*A*

In [96]:
*a*

#<ATOM :HH33/:H>

In [97]:
(:= *p* (core:make-cxx-object 'chem:chem-info))

*P*

In [98]:
(chem:compile-smarts *p* "CC")

T

In [99]:
(chem:matches *p* *a*)

NIL